In [1]:
import torch

In [3]:
from configs.custom_config import load_config
from slowfast.utils.misc import launch_job
from slowfast.utils.parser import parse_args
import numpy as np
import torch
import os
import time
from tqdm import tqdm
import av
from moviepy.video.io.VideoFileClip import VideoFileClip

import slowfast.utils.checkpoint as cu
import slowfast.utils.distributed as du
import slowfast.utils.logging as logging
import slowfast.utils.misc as misc

from models import build_model
from datasets.extract_dataset import VideoSet




In [6]:
def build_model(cfg):
    """
    Builds the video model.
    The original function has been customized to load the feature extractor models with overwritten forward functions
    Args:
        cfg (configs): configs that contains the hyper-parameters to build the
        backbone. Details can be seen in slowfast/config/defaults.py.
    """
    if torch.cuda.is_available():
        assert (
            cfg.NUM_GPUS <= torch.cuda.device_count()
        ), "Cannot use more GPU devices than available"
    else:
        assert (
            cfg.NUM_GPUS == 0
        ), "Cuda is not available. Please set `NUM_GPUS: 0 for running on CPUs."

    # Construct the model
    name = cfg.MODEL.MODEL_NAME
    # model = MODEL_REGISTRY.get(name)(cfg)

    # load feature extractor models
    width_per_group = cfg.RESNET.WIDTH_PER_GROUP
    pool_size = _POOL1[cfg.MODEL.ARCH]
    if name == "SlowFast":
        model = SlowFastFeat(cfg)
        model.head = ResNetBasicHead(
            dim_in=[
                width_per_group * 32,
                width_per_group * 32 // cfg.SLOWFAST.BETA_INV,
            ],
            num_classes=cfg.MODEL.NUM_CLASSES,
            pool_size=[None, None]
            if cfg.MULTIGRID.SHORT_CYCLE
            else [
                [
                    cfg.DATA.NUM_FRAMES // cfg.SLOWFAST.ALPHA // pool_size[0][0],
                    cfg.DATA.TEST_CROP_SIZE // 32 // pool_size[0][1],
                    cfg.DATA.TEST_CROP_SIZE // 32 // pool_size[0][2],
                ],
                [
                    cfg.DATA.NUM_FRAMES // pool_size[1][0],
                    cfg.DATA.TEST_CROP_SIZE // 32 // pool_size[1][1],
                    cfg.DATA.TEST_CROP_SIZE // 32 // pool_size[1][2],
                ],
            ],  # None for AdaptiveAvgPool3d((1, 1, 1))
            dropout_rate=cfg.MODEL.DROPOUT_RATE,
            act_func=cfg.MODEL.HEAD_ACT,
            cfg=cfg
        )
    elif name == "ResNet":
        model = ResNetFeat(cfg)
        model.head = ResNetBasicHead(
            dim_in=[width_per_group * 32],
            num_classes=cfg.MODEL.NUM_CLASSES,
            pool_size=[None, None]
            if cfg.MULTIGRID.SHORT_CYCLE
            else [
                [
                    cfg.DATA.NUM_FRAMES // pool_size[0][0],
                    cfg.DATA.CROP_SIZE // 32 // pool_size[0][1],
                    cfg.DATA.CROP_SIZE // 32 // pool_size[0][2],
                ]
            ],  # None for AdaptiveAvgPool3d((1, 1, 1))
            dropout_rate=cfg.MODEL.DROPOUT_RATE,
            act_func=cfg.MODEL.HEAD_ACT,
            
        )

    if cfg.NUM_GPUS:
        # Determine the GPU used by the current process
        cur_device = cfg.USED_GPU
        # Transfer the model to the current GPU device
        model = model.cuda(device=cur_device)
    # Use multi-process data parallel model in the multi-gpu setting
    if cfg.NUM_GPUS > 1:
        # Make model replica operate on the current device
        model = torch.nn.parallel.DistributedDataParallel(
            module=model, device_ids=[cur_device], output_device=cur_device
        )
    return model



usage: ipykernel_launcher.py [-h] [--shard_id SHARD_ID]
                             [--num_shards NUM_SHARDS]
                             [--init_method INIT_METHOD]
                             [--cfg CFG_FILES [CFG_FILES ...]] [--opts ...]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"669ba70c-be02-4d55-b26e-5bc77ff9ce56" --shell=9002 --transport="tcp" --iopub=9004 --f=/home/pangy/.local/share/jupyter/runtime/kernel-v2-9351iDYGvM00xGSr.json


SystemExit: 2